In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.metrics import mean_squared_error
from scipy.ndimage.interpolation import shift
from math import sin, cos, sqrt, atan2, radians 
from sklearn import tree, svm, linear_model, ensemble, neighbors, naive_bayes 
import dateutil
import os
from pathlib import Path
from joblib import dump, load
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import make_scorer


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
status_path = Path('../filtered_status.csv')
stations_path = Path('../station.csv')

In [3]:
status_df = pd.read_csv(status_path, parse_dates=['time'])
stations_df = pd.read_csv(stations_path)

In [4]:
status_df

,station_id,bikes_available,docks_available,time
0,2,2,25,2013-08-29 12:06:01
1,2,2,25,2013-08-29 12:07:01
2,2,2,25,2013-08-29 12:08:01
3,2,2,25,2013-08-29 12:09:01
4,2,2,25,2013-08-29 12:10:01
...,...,...,...,...
71977905,84,8,7,2015-08-31 23:55:02
71977906,84,8,7,2015-08-31 23:56:01
71977907,84,8,7,2015-08-31 23:57:02
71977908,84,8,7,2015-08-31 23:58:02


In [5]:
stations_df

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013
...,...,...,...,...,...,...,...
65,77,Market at Sansome,37.789625,-122.400811,27,San Francisco,8/25/2013
66,80,Santa Clara County Civic Center,37.352601,-121.905733,15,San Jose,12/31/2013
67,82,Broadway St at Battery St,37.798541,-122.400862,15,San Francisco,1/22/2014
68,83,Mezes Park,37.491269,-122.236234,15,Redwood City,2/20/2014


## Ottengo gli id delle stazioni per ogni città diversa

In [5]:
cities = stations_df['city'].unique()
cities

array(['San Jose', 'Redwood City', 'Mountain View', 'Palo Alto',
       'San Francisco'], dtype=object)

In [6]:
city_stations = {}

for city in cities:
    city_stations[city] = stations_df[stations_df['city'] == city]['id'].unique()

city_stations

{'San Jose': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 80, 84]),
 'Redwood City': array([21, 22, 23, 24, 25, 26, 83]),
 'Mountain View': array([27, 28, 29, 30, 31, 32, 33]),
 'Palo Alto': array([34, 35, 36, 37, 38]),
 'San Francisco': array([41, 42, 45, 46, 47, 48, 49, 50, 51, 39, 54, 55, 56, 57, 58, 59, 60,
        61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
        82])}

## Prendo in cosiderazione solo gli id delle stazioni appartenenti alla città di San Francisco

In [7]:
San_Fancisco_stations = city_stations["San Francisco"]
San_Fancisco_stations

array([41, 42, 45, 46, 47, 48, 49, 50, 51, 39, 54, 55, 56, 57, 58, 59, 60,
       61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       82])

In [8]:
interval=30
window_width = 5
drop_bikes_inf = True

## Training

In [8]:
classifiers = {
    'DecisionTree': tree.DecisionTreeClassifier(random_state=42),   
#     'SVC': svm.SVC(),
#     'LogisticRegression': linear_model.LogisticRegression(random_state=42, solver='liblinear', max_iter=1000),
#     'NaiveBayes': naive_bayes.GaussianNB(),
#     'RandomForest': ensemble.RandomForestClassifier(random_state=42, max_features='sqrt'),
}

Effettuo il finetuning dei modelli tramite la grid search

In [11]:
decision_tree_parameters = {
    'max_depth': list(range(2, 7)),
    'min_samples_split': list(range(2, 7)),
    'criterion' : ['gini', 'entropy'],
    'class_weight': [None, 'balanced']
}

SVC_parameters = {
    'C': [0.1, 1, 100],
    'gamma': [0.1, 0.01, 0.001],
    'kernel': ['rbf'],
    'class_weight': [None, 'balanced']
}

logistic_regression_parameters = {
    'penalty': ['l1', 'l2'],
    'C': [1, 0.1, 0.01, 0.001],
    'class_weight': [None, 'balanced']
}

random_forest_parameters = {
    'n_estimators': [10, 100, 1000],
    'criterion' : ['gini', 'entropy'],
    'max_depth': list(range(0, 5)), 
    'min_samples_split': list(range(0, 5)), 
    'class_weight': [None, 'balanced']
}

naive_bayes_parameters = {
    'var_smoothing': np.logspace(0,-9, num=100)
}

classifiers_parameters = {
  'DecisionTree': decision_tree_parameters,
  'SVC': SVC_parameters,
  'LogisticRegression': logistic_regression_parameters,
  'RandomForest': random_forest_parameters,
  'NaiveBayes': naive_bayes_parameters
}

tuned_classifiers = {
    'DecisionTree': [],
    'SVC': [],
    'LogisticRegression': [],
    'RandomForest': [],
    'KNN': []
}

for reference_station in San_Fancisco_stations:
    train_df = pd.read_csv(f"./datasets/{interval}_{window_width}/station{reference_station}_train.csv")
    
    # se il flag è a true, rimuovo tutte le informazioni relative alle bici disponibili
    if drop_bikes_inf:
        train_df = train_df[train_df.columns.drop(list(train_df.filter(regex='bikes')))]
        
    y = train_df['status']
    X = train_df.drop(columns=['status'])

    for scoring in ['accuracy', 'recall', 'precision', 'f1_score']:
        for classifier in classifiers.keys():
            print(f"Classifier: {classifier}" )   
        
            clf = classifiers[classifier]
            parametrers = classifiers_parameters[classifier]
            
            #I segunti controlli sono necessari in quanto richiamo, precisione e 
            # f1score hanno necessità di conoscere quale sia la classe positiva (QP)
            if scoring == "recall":
                scorer = make_scorer(recall_score, pos_label="QP")
            elif scoring == "precision":
                scorer = make_scorer(precision_score, pos_label="QP")
            elif scoring == "f1_score":
                scorer = make_scorer(f1_score, pos_label="QP")
            else:
                scorer = scoring
            
            #Nel caso di logist regression utilizzo la randomizedSearchCV, in quanto più veloce rispetto alla gridSearchCV
            if classifier == 'LogisticRegression':
                search_result = RandomizedSearchCV(clf, parametrers, scoring=scorer, random_state=42, n_iter=5, n_jobs=-1)
            else:
                search_result = GridSearchCV(clf, parametrers, scoring=scorer, cv=3, n_jobs=-1)
                
            fit_result = search_result.fit(X, y)
            params = fit_result.best_params_

            print("-"*10)
            print(f"Best parameters for classifier {classifier} for the station {reference_station} using the scoring {scoring}: ")
            print(params)
            print()
            print()

            tuned_clf = search_result.best_estimator_
            
            #salvo il modello dopo il fine tuning
            if drop_bikes_inf:
                if (Path.cwd() / f'tuned_models_wo_bikes_inf/{scoring}/{classifier}/{interval}_{window_width}').exists() == False:
                    os.makedirs(f"./tuned_models_wo_bikes_inf/{scoring}/{classifier}/{interval}_{window_width}")
                dump(tuned_clf, f'./tuned_models_wo_bikes_inf/{scoring}/{classifier}/{interval}_{window_width}/station{reference_station}_tuned_model.joblib')
            else:
                if (Path.cwd() / f'tuned_models/{scoring}/{classifier}/{interval}_{window_width}').exists() == False:
                    os.makedirs(f"./tuned_models/{scoring}/{classifier}/{interval}_{window_width}")
                dump(tuned_clf, f'./tuned_models/{scoring}/{classifier}/{interval}_{window_width}/station{reference_station}_tuned_model.joblib')

Classifier: DecisionTree
----------
Best parameters for classifier DecisionTree for the station 41 using the scoring accuracy: 
{'class_weight': None, 'criterion': 'entropy', 'max_depth': 2, 'min_samples_split': 2}


Classifier: DecisionTree
----------
Best parameters for classifier DecisionTree for the station 41 using the scoring recall: 
{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_split': 2}


Classifier: DecisionTree
----------
Best parameters for classifier DecisionTree for the station 41 using the scoring precision: 
{'class_weight': None, 'criterion': 'entropy', 'max_depth': 2, 'min_samples_split': 2}


Classifier: DecisionTree
----------
Best parameters for classifier DecisionTree for the station 41 using the scoring f1_score: 
{'class_weight': None, 'criterion': 'entropy', 'max_depth': 2, 'min_samples_split': 2}


Classifier: DecisionTree
----------
Best parameters for classifier DecisionTree for the station 42 using the scoring accuracy: 
{'

## Testing dei fine tuned models

In [ ]:

results_table = {}
thresholds = [0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.95]

for scoring in ["precision"]:
    result_matrix = []
    for classifier in classifiers.keys():
        for threshold in thresholds:
            
            if drop_bikes_inf:
                file = open(f'test_results/tuned_classifiers_with_threshold/without_bikes_inf/{classifier}_{scoring}_{interval}_{window_width}_({int(threshold*100)}%)_results.txt', "w")
            else:
                file = open(f'test_results/tuned_classifiers_with_threshold/{classifier}_{scoring}_{interval}_{window_width}_({int(threshold*100)}%)_results.txt', "w")
            file.write(f'TESING RESULTS FOR {classifier} CLASSIFIER WITH THRESHOLD {int(threshold*100)}%:\n\n')
            tot_fp = 0
            tot_tp = 0
            tot_fn = 0
            tot_tn = 0

            for station_id in San_Fancisco_stations:
                
                if drop_bikes_inf:
                    model = load(f'./tuned_models_wo_bikes_inf/{scoring}/{classifier}/{interval}_{window_width}/station{station_id}_tuned_model.joblib')
                else:
                    model = load(f'./tuned_models/{scoring}/{classifier}/{interval}_{window_width}/station{station_id}_tuned_model.joblib')
                
                test_df = pd.read_csv(f'./datasets/{interval}_{window_width}/station{station_id}_test.csv')
                
                #se il flag è true, rimuovo le info sulle biciclette
                if drop_bikes_inf:
                    test_df = test_df[test_df.columns.drop(list(test_df.filter(regex='bikes')))]
                
                y_test = test_df['status']
                X_test = test_df.drop(columns=['status'])    

#                 pred = model.predict(X_test)
#                 print (pred)

                prediction_proba = model.predict_proba(X_test)
                # prediction_proba è una matrice con due colonne: la prima colonna contiene le probabilità che il sample
                # appartenga alla classe "N", il secondo contiene la probabilità di appartenenza alla classe "QP".
                # quindi considerero la seconda colonna.

                prediction = []
                for el in prediction_proba:
                    if el[1] >= threshold:
                        prediction.append("QP")
                    else:
                        prediction.append("N")

                cm = confusion_matrix(y_test, prediction)

                str_= f'{classifier} FOR STATION {station_id}' + '\n'
                str_ += f'Confusion matrix:' + '\n'
                str_ += str(cm) + '\n'

                tn, fp, fn, tp = cm.ravel()
                str_+= f'tp={tp}, fn={fn}, fp={fp}, tn={tn}' +'\n'

                test_accuracy = (tn + tp) / (tn + fp + fn + tp)
    #             test_recall = (tp) / (tp + fn)
    #             test_precision = (tp) / (tp + fp)
                test_recall = recall_score(y_test, prediction, pos_label='QP')
                test_precision = precision_score(y_test, prediction, pos_label='QP', zero_division=0)
                test_f1_score = f1_score(y_test, prediction, pos_label='QP')

                str_+= f'accuracy={test_accuracy}; recall={test_recall}; precision={test_precision}; f1_score= {test_f1_score}' +'\n\n'
                str_+= "-"*10 +'\n\n'

                tot_fp += fp
                tot_tp += tp
                tot_fn += fn
                tot_tn += tn

                file.write(str_)

            avg_accuracy = (tot_tn + tot_tp) / (tot_tn + tot_fp + tot_fn + tot_tp)
            avg_recall = (tot_tp) / (tot_tp + tot_fn)
            avg_precision = (tot_tp) / (tot_tp + tot_fp)
            avg_f1_score = 2*(1/((1/avg_recall)+(1/avg_precision)))

            result_matrix.append([avg_accuracy, avg_recall, avg_precision, avg_f1_score])
            avg_str = f"AVERAGE VALUES FOR {classifier}: accuracy={avg_accuracy}; recall={avg_recall}; precision={avg_precision}; f1_score={avg_f1_score}"

            file.write(avg_str)
            file.close()

    result_table = pd.DataFrame(result_matrix, columns=['avg_accuracy', 'avg_recall', 'avg_precision', 'avg_f1_score'], index=pd.Index(thresholds))
    results_table[scoring] = result_table
    if drop_bikes_inf:
        result_table.to_csv(f"./test_results/tuned_classifiers_with_threshold/without_bikes_inf/Overall_results_{scoring}_{interval}_{window_width}.csv")
    else:
        result_table.to_csv(f"./test_results/tuned_classifiers_with_threshold/Overall_results_{scoring}_{interval}_{window_width}.csv")


In [6]:
rows = []
result_matrix = []
columns = ["avg_recall", "avg_precision", "avg_f1_score"]

curr_drop_bikes_inf = True

thresholds = [0.5, 0.6, 0.7, 0.8, 0.85, 0.9]
current_scoring = "precision"
current_interval = 30
current_window_width = 5


for threshold in thresholds:
    
    input_file_name = f"DecisionTree_{current_scoring}_{current_interval}_{current_window_width}_({int(threshold*100)}%)_results.txt"

    if curr_drop_bikes_inf:   
        input_file_path = f"./test_results/tuned_classifiers_with_threshold/without_bikes_inf/{input_file_name}"
    else:
        input_file_path = f"./test_results/tuned_classifiers_with_threshold/{input_file_name}"
        
    with open(input_file_path, "r") as file:
        lines = file.read().splitlines()
        last_line = lines[-1]
#                         print (last_line)
        results = last_line.split(';')
        result_values = []
        for result in results:
            value = round(float(result.split('=')[1]), 5)
            result_values.append(value)
        result_matrix.append(result_values[1:])

results_table = pd.DataFrame(result_matrix, columns=columns, index=pd.Index(thresholds))
# print(results_table.to_latex(float_format="%.4f", bold_rows=True))
results_table

,avg_recall,avg_precision,avg_f1_score
0.50,0.73314,0.77152,0.75184
0.60,0.65806,0.79916,0.72178
0.70,0.60791,0.81685,0.69706
0.80,0.47587,0.84182,0.60803
0.85,0.36756,0.84898,0.51301
0.90,0.04768,0.90943,0.09060


In [5]:
#Codice per ottenere il latex delle diverse tabelle
print(results_table.to_latex(bold_rows=True))

\begin{tabular}{lrrr}
\toprule
{} &  avg\_recall &  avg\_precision &  avg\_f1\_score \\
\midrule
\textbf{0.50} &     0.72433 &        0.77377 &       0.74824 \\
\textbf{0.60} &     0.65124 &        0.80410 &       0.71964 \\
\textbf{0.70} &     0.60544 &        0.82294 &       0.69763 \\
\textbf{0.80} &     0.48981 &        0.84650 &       0.62055 \\
\textbf{0.85} &     0.38061 &        0.85625 &       0.52698 \\
\textbf{0.90} &     0.05490 &        0.92040 &       0.10361 \\
\bottomrule
\end{tabular}



In [16]:
curr_drop_bikes_inf = False

# stampo i risultati ottenuti leggendoli nel file csv salvato in caso di mancanza di bici

if curr_drop_bikes_inf:
    res = pd.read_csv(f"./test_results/tuned_classifiers_with_threshold/without_bikes_inf/Overall_results_precision_30_5.csv", index_col=0)

else:
    res = pd.read_csv(f"./test_results/tuned_classifiers_with_threshold/without_bikes_inf/Overall_results_precision_30_5.csv", index_col=0)
res = res.drop(columns = ['avg_accuracy'])
res = res.drop([0.95])
res

,avg_recall,avg_precision,avg_f1_score
0.50,0.733136,0.771521,0.751839
0.60,0.658061,0.799159,0.721779
0.70,0.607913,0.816853,0.697062
0.80,0.475865,0.841820,0.608025
0.85,0.367557,0.848983,0.513012
0.90,0.047676,0.909434,0.090602


In [14]:
print("PRECISION FOR DECISION TREE")
results_table["precision"]

PRECISION FOR DECISION TREE


,avg_accuracy,avg_recall,avg_precision,avg_f1_score
0.50,0.982899,0.733136,0.771521,0.751839
0.60,0.982074,0.658061,0.799159,0.721779
0.70,0.981330,0.607913,0.816853,0.697062
0.80,0.978321,0.475865,0.841820,0.608025
0.85,0.975343,0.367557,0.848983,0.513012
0.90,0.966183,0.047676,0.909434,0.090602
0.95,0.966032,0.042136,0.924078,0.080598


In [23]:
print("PRECISION FOR DECISION TREE")
results_table["precision"]

PRECISION FOR DECISION TREE


,avg_accuracy,avg_recall,avg_precision,avg_f1_score
0.50,0.982777,0.724332,0.773774,0.748237
0.60,0.982071,0.651236,0.804104,0.719641
0.70,0.981456,0.605440,0.822936,0.697629
0.80,0.978834,0.489812,0.846496,0.620551
0.85,0.975857,0.380613,0.856253,0.526979
0.90,0.966438,0.054896,0.920398,0.103612
0.95,0.966018,0.041048,0.936795,0.078651


In [20]:
print("F1_SCORE FOR DECISION TREE")
results_table["f1_score"]

F1_SCORE FOR DECISION TREE


,avg_accuracy,avg_recall,avg_precision,avg_f1_score
0.50,0.980736,0.768942,0.709954,0.738272
0.60,0.980176,0.702572,0.727170,0.714659
0.70,0.979429,0.597527,0.768771,0.672418
0.80,0.977066,0.490406,0.778580,0.601772
0.85,0.975374,0.422255,0.779868,0.547870
0.90,0.969265,0.203165,0.735673,0.318400


### Ricavo la sequenza di regole che hanno determinato la predizione di quei record che sono classificati come “QuasiPiena” con un certa soglia di probabilità

In [9]:
prob_threshold = 90

file1 = open(f'test_results/tuned_classifiers_with_threshold/Paths_DecisionTree_precision_30_5_({prob_threshold}%).txt', "w")
file1.write(f'DECISION PATHS FOR DECISION TREE CLASSIFIER WITH THRESHOLD {prob_threshold}%:\n\n')

for station_id in San_Fancisco_stations:
    positive_records = pd.DataFrame()
    model = load(f'./tuned_models/precision/DecisionTree/30_5/station{station_id}_tuned_model.joblib')
    feature = model.tree_.feature
    threshold = model.tree_.threshold
    test_df = pd.read_csv(f'./datasets/{interval}_{window_width}/station{station_id}_test.csv')
    y_test = test_df['status']
    X_test = test_df.drop(columns=['status'])    
#     plt.figure(figsize=(80,40))
#     tree.plot_tree(model, fontsize=10, feature_names=X_test.columns, class_names=['N', 'QP'], label='all', filled=True, rounded=True)
    
    prediction_proba = model.predict_proba(X_test)
    # prediction_proba è una matrice con due colonne: la prima colonna contiene le probabilità che il sample
    # appartenga alla classe "N", il secondo contiene la probabilità di appartenenza alla classe "QP".
    # quindi considerero la seconda colonna.

    for index, el in enumerate(prediction_proba):
        if el[1] >= prob_threshold/100: 
            positive_records = positive_records.append(X_test.iloc[[index]], ignore_index=True)
    
    if len (positive_records) >0:
        
        file1.write(f"------------------------   STATION {station_id}   -----------------------------\n\n")
        print(positive_records.shape)
        
        for sample_id, row in positive_records.iterrows():
            node_indicator = model.decision_path(positive_records)
            leaf_id = model.apply(positive_records)
            
            # obtain ids of the nodes `sample_id` goes through, i.e., row `sample_id`
            node_index = node_indicator.indices[
                node_indicator.indptr[sample_id] : node_indicator.indptr[sample_id + 1]
            ]

            file1.write("Regole usate per predire il record {id}:\n".format(id=sample_id))
            for node_id in node_index:
                # continue to the next node if it is a leaf node
                if leaf_id[sample_id] == node_id:
                    continue
                
                # check if value of the split feature for sample 0 is below threshold
                if positive_records.iloc[sample_id].values[feature[node_id]] <= threshold[node_id]:
                    threshold_sign = "<="
                else:
                    threshold_sign = ">"

                file1.write(
                    "decision node {node} : ({feature} = {value}) "
                    "{inequality} {threshold}\n".format(
                        node=node_id,
                        sample=sample_id,
                        feature=positive_records.columns[feature[node_id]],
                        value=positive_records.iloc[sample_id].values[feature[node_id]],
                        inequality=threshold_sign,
                        threshold=threshold[node_id],
                    )
                )
            file1.write("\n")
            
file1.close()   

(16, 350)
(78, 350)
(4, 350)
(2, 350)
(144, 350)
(65, 350)
(294, 350)


In [ ]:
# !zip -r ../Zips/Classification.zip ../Classification